<a href="https://colab.research.google.com/github/rrsartneoai/Awesome-Multimodal-Large-Language-Models/blob/main/Kopia_notatnika_finetune_rafal_stepkowski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install pyarrow==14.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires pyarrow>=15.0.0, but you have pyarrow 14.0.1 which is incompatible.


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset('text', data_files={'train': 'train.txt', 'test': 'test.txt'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Inicjalizacja tokenizer'a i modelu

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenizacja danych

def tokenize_function(examples):
    # Indent the return statement to be part of the function
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Parametry fine-tuningu

training_args = TrainingArguments(
output_dir='./results',
num_train_epochs=3, # Liczba epok
per_device_train_batch_size=16, # Batch size
per_device_eval_batch_size=16,
learning_rate=2e-5, # Learning rate
eval_strategy="epoch", # updated from evaulation to eval - rafal
logging_dir='./logs',
)

In [ ]:
# Inicjalizacja i uruchomienie fine-tuningu - Rafal
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import load_metric

# Load the metric
metric = load_metric("accuracy")  # Or any other suitable metric

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].add_column('labels', [0]*len(tokenized_datasets["train"])),  # Add dummy labels for demonstration
    eval_dataset=tokenized_datasets["test"].add_column('labels', [0]*len(tokenized_datasets["test"])),  # Add dummy labels for demonstration
    compute_metrics=compute_metrics,
)

# Start the training process
trainer.train()

# Potencjalne problemy i ulepszenia
Na podstawie ograniczonych informacji można wskazać następujące potencjalne problemy i obszary wymagające poprawy:

Brak poprawy utraty walidacji : Utrata walidacji wzrosła od drugiej do trzeciej epoki, co wskazuje na potencjalne nadmierne dopasowanie. Można temu zaradzić za pomocą technik, takich jak wczesne zatrzymanie, regularyzacja lub zwiększenie różnorodności zestawu danych.
Niska dokładność : Ostateczna dokładność 0,2 jest dość niska. Może to być spowodowane różnymi czynnikami, w tym niewystarczającymi danymi, słabą architekturą modelu lub nieodpowiednim dostrajaniem hiperparametrów. Eksperymentuj z różnymi modelami, hiperparametrami i technikami powiększania danych.
Brak rejestrowania : Brak rejestrowania może utrudniać szczegółową analizę procesu szkolenia. Rozważ włączenie rejestrowania, aby śledzić metryki, takie jak wskaźniki uczenia się, gradienty i inne istotne informacje.
Czy chciałbyś zagłębić się w jakiś konkretny aspekt procesu szkoleniowego lub zbadać potencjalne rozwiązania wspomnianych problemów?
Proszę, podaj więcej kontekstu lub zadaj konkretne pytania.

<ipython-input-9-088fd6a0fa69>:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")  # Or any other suitable metric


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.937880,0.000000
2,No log,0.838561,0.000000
3,No log,0.780368,0.200000


TrainOutput(global_step=3, training_loss=0.899137814839681, metrics={'train_runtime': 11.5718, 'train_samples_per_second': 2.593, 'train_steps_per_second': 0.259, 'total_flos': 7893331660800.0, 'train_loss': 0.899137814839681, 'epoch': 3.0})